In [22]:
import math
import warnings
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

warnings.filterwarnings("ignore")
print(tf.__version__)

2.17.0


# Data Pre-processing

In [23]:
raw = pd.read_csv("scats-10-2006.csv")
raw.describe()

,SCATS Number,NB_LATITUDE,NB_LONGITUDE,HF VicRoads Internal,VR Internal Stat,VR Internal Loc,NB_TYPE_SURVEY,V00,V01,V02,...,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95
count,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.0,4192.000000,4192.000000,4192.000000,...,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000,4192.000000
mean,3639.151718,-37.542437,143.985065,10234.837786,819.153626,4.252147,1.0,34.049380,30.185592,25.936307,...,76.283874,73.049380,72.192748,65.655057,61.398378,56.124761,52.262643,45.383111,40.827529,35.408158
std,805.584115,3.240889,12.429455,5579.371363,628.218870,2.194029,0.0,28.438688,26.565710,24.243348,...,37.577091,36.049374,36.669158,33.885956,33.121580,32.505183,33.070764,30.096586,30.012899,27.518453
min,970.000000,-37.867600,0.000000,-1.000000,182.000000,1.000000,1.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3122.000000,-37.833120,145.036169,5887.000000,380.000000,3.000000,1.0,15.000000,12.000000,10.000000,...,51.000000,48.000000,46.000000,42.000000,37.000000,33.000000,29.000000,24.000000,20.000000,17.000000
50%,4032.000000,-37.822846,145.058038,10074.000000,650.000000,5.000000,1.0,26.000000,22.000000,18.000000,...,70.000000,67.000000,66.000000,60.000000,55.000000,50.000000,45.000000,39.000000,33.000000,28.000000
75%,4263.000000,-37.808920,145.077826,16149.000000,950.000000,7.000000,1.0,44.000000,39.000000,33.000000,...,97.000000,95.000000,93.000000,85.000000,81.000000,74.000000,68.000000,59.000000,53.000000,46.000000
max,4821.000000,0.000000,145.098850,20314.000000,2707.000000,8.000000,1.0,304.000000,252.000000,201.000000,...,334.000000,259.000000,275.000000,275.000000,231.000000,256.000000,324.000000,221.000000,279.000000,212.000000


We have 4192 values to work with.

Let's first, based on description, decide what data will be relevant:

- **SCATS Number**: ID of the intersection. We will be using this as part of our predictor.
- **Location**: Direction of the intersection. Will be hard to use this, so discard.
- **CD_MELWAY**: Essentially same as SCATS number, but in a harder to use form. Discard.
- **NB_LATITUDE, NB_LONGITUDE**: Coordinates of the intersection. Changes depending on direction (N, E, S, W, etc). For now, maybe not relevant, so discard
- **HF VicRoads Internal**: This seems to point to specific intersection exits. Could be useful?
- **VR Internal Stat/Loc**: Internal Stat seems to point to intersections, while Loc seems to point in cardinal directions as a numeral. Stat represents the same data as SCATS number, but Loc could be useful? (Similar to HF VicRoads Internal)
- **NB_TYPE_SURVEY**: Same for every entry, discard.
- **Date**: Date in dd/MM/YYYY format. As this data is only for October of 2006, the actual date might not be useful. We can try representing this as day of the week instead? This might yield more meaningful results.
- **V00 - V95**: describes times from 00:00 - 23:45, in 15-minute intervals. This is what we will use as a predictor.

In [3]:
raw.head()

,SCATS Number,Location,CD_MELWAY,NB_LATITUDE,NB_LONGITUDE,HF VicRoads Internal,VR Internal Stat,VR Internal Loc,NB_TYPE_SURVEY,Date,...,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95
0,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,1/10/2006,...,114,97,97,66,81,50,59,47,29,34
1,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,2/10/2006,...,111,102,107,114,80,60,62,48,44,26
2,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,3/10/2006,...,130,132,114,86,93,90,73,57,29,40
3,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,4/10/2006,...,115,113,132,101,113,90,78,66,52,44
4,970,WARRIGAL_RD N of HIGH STREET_RD,060 G10,-37.86703,145.09159,249,182,1,1,5/10/2006,...,171,120,116,113,99,91,61,55,49,36


For a preliminary training run, we will focus on data from only one intersection.

In [4]:
# Drop the fields we've decided to discard
df = raw.drop(["NB_LATITUDE", "NB_LONGITUDE", "CD_MELWAY", 
                 "Location", "HF VicRoads Internal", "VR Internal Stat", 
                 "VR Internal Loc", "NB_TYPE_SURVEY"], axis=1)
df.head()

,SCATS Number,Date,V00,V01,V02,V03,V04,V05,V06,V07,...,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95
0,970,1/10/2006,86,83,52,58,59,44,31,37,...,114,97,97,66,81,50,59,47,29,34
1,970,2/10/2006,32,28,17,11,7,11,6,15,...,111,102,107,114,80,60,62,48,44,26
2,970,3/10/2006,26,32,21,14,10,12,13,10,...,130,132,114,86,93,90,73,57,29,40
3,970,4/10/2006,32,22,28,13,16,8,14,10,...,115,113,132,101,113,90,78,66,52,44
4,970,5/10/2006,40,39,21,11,16,9,15,15,...,171,120,116,113,99,91,61,55,49,36


Our time of day is scattered, so let's convert the V00 - V95 columns into their own rows with a new column: 'V', which will represent the 15-minute increment to predict. Date will be replaced with day of the week.

In [5]:
import datetime

# parse d/MM/YYYY format to DateTime
def parse_date(date_string):
    split = date_string.split('/')

    if len(split) != 3:
        print(f"invalid date format: {date_string}")

    day = int(split[0])
    month = int(split[1])
    year = int(split[2])

    return datetime.datetime(year, month, day)

In [24]:
def append_v(n):
    if n < 10:
        column_name = "V0" + str(n)
    else:
        column_name = "V" + str(n)
        
    return column_name

In [25]:
unique_scats = df['SCATS Number'].unique()
print(f"{len(unique_scats)} unique SCATS numbers.")

40 unique SCATS numbers.


In [28]:
for scats_n in unique_scats:
    print(f"Processing SCATS Number: {scats_n}")
    
    data = []
    
    for index, row in df[df['SCATS Number'] == scats_n].iterrows():
        for i in range(0, 96):
            col = append_v(i)
            data.append(row[col])

    processed = pd.DataFrame(data, columns=['VFlow'])

    row_split = int(processed.shape[0] * 0.7)
    print(row_split) 
    train_dataset = processed[:row_split]
    test_dataset = processed[row_split:]

    train_filename = f"intersection/train/train_{scats_n}.csv"
    test_filename = f"intersection/test/test_{scats_n}.csv"
    
    train_dataset.to_csv(train_filename, index=False)
    test_dataset.to_csv(test_filename, index=False)
    
    print(f"Saved train data to {train_filename} and test data to {test_filename}")


Processing SCATS Number: 970
8265
Saved train data to intersection/train/train_970.csv and test data to intersection/test/test_970.csv
Processing SCATS Number: 2000
7660
Saved train data to intersection/train/train_2000.csv and test data to intersection/test/test_2000.csv
Processing SCATS Number: 2200
8332
Saved train data to intersection/train/train_2200.csv and test data to intersection/test/test_2200.csv
Processing SCATS Number: 2820
4099
Saved train data to intersection/train/train_2820.csv and test data to intersection/test/test_2820.csv
Processing SCATS Number: 2825
2083
Saved train data to intersection/train/train_2825.csv and test data to intersection/test/test_2825.csv
Processing SCATS Number: 2827
8332
Saved train data to intersection/train/train_2827.csv and test data to intersection/test/test_2827.csv
Processing SCATS Number: 2846
8332
Saved train data to intersection/train/train_2846.csv and test data to intersection/test/test_2846.csv
Processing SCATS Number: 3001
6384
Sa

## note: functions below this line are deprecated
converts data straight to time lag form, which is NOT used by the repo, thus should be disregarded

In [29]:
lag = 6 

for scats_n in unique_scats:
    data = []
    
    for index, row in df[df['SCATS Number'] == scats_n].iterrows():
        for i in range(lag, 96):
            lag_cols = []
            for j in range(lag, 0, -1):
                lag_cols.append(append_v(i - j))
            col = append_v(i)
            
            data.append([row[lag_cols].values, row[col]])

    processed_lagged = pd.DataFrame(data, columns=[f'Last{lag}VFlow', 'VFlow'])
    
    processed_lagged_filename = f"intersection/processed/{scats_n}_processed.csv"
    processed_lagged.to_csv(processed_lagged_filename, index=False)
    
    print(f"Processed lagged data saved for SCATS {scats_n} to {processed_lagged_filename}")


Processed lagged data saved for SCATS 970 to intersection/processed/970_processed.csv
Processed lagged data saved for SCATS 2000 to intersection/processed/2000_processed.csv
Processed lagged data saved for SCATS 2200 to intersection/processed/2200_processed.csv
Processed lagged data saved for SCATS 2820 to intersection/processed/2820_processed.csv
Processed lagged data saved for SCATS 2825 to intersection/processed/2825_processed.csv
Processed lagged data saved for SCATS 2827 to intersection/processed/2827_processed.csv
Processed lagged data saved for SCATS 2846 to intersection/processed/2846_processed.csv
Processed lagged data saved for SCATS 3001 to intersection/processed/3001_processed.csv
Processed lagged data saved for SCATS 3002 to intersection/processed/3002_processed.csv
Processed lagged data saved for SCATS 3120 to intersection/processed/3120_processed.csv
Processed lagged data saved for SCATS 3122 to intersection/processed/3122_processed.csv
Processed lagged data saved for SC